# Classifying 3x3 mazes - Without using symmetry
In this notebook we use the same Quantum Machine Learing approach as for the Tic-Tac-Toe boards, but for a different application: determining whether a path exists between any two corners of a 3x3 maze. This time, we do not make use of the symmetry of the problem, and compare both outcomes.

In [ ]:
from qiskit import *
import numpy as np
import maze as m

from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

def make_circuit(x, params, layers):
    circ = QuantumCircuit(9)
    for i in range(layers):
        l = i*34
        m.encode_data(x, circ)
        m.ns_add_single_qubit_gates(params[l:l+18], circ)
        m.ns_add_two_qubit_gates(params[l+18:l+34], circ)

    return circ
# Create a Quantum Circuit acting on a quantum register of nine qubits

layers = 4

params = np.random.rand(34*layers)*2*np.pi
circ = make_circuit([1,1,0,0,0,1,0,0,0], params, layers)

maze_fields, grid_combinations = m.generate_maze_field()

In [20]:
# Define the loss function
def l2_loss(output, target):
    output, target = np.array(output), np.array(target)
    return np.abs(output - target)**2

In [21]:
x = maze_fields
y = m.check(grid_combinations)
#print(x)
#print(y)

# shuffle the indices
shuffle_indices = np.random.permutation(len(x))
train_size = int(len(x) * 0.3)

# split the indices into training and testing sets
train_indices = np.array(shuffle_indices[:train_size])
test_indices = np.array(shuffle_indices[train_size:])

# create the training and testing sets
x_train, y_train = np.take(x, train_indices, axis=0), np.take(y, train_indices, axis=0)
x_test, y_test = np.take(x, test_indices, axis=0), np.take(y, test_indices, axis=0)


print("Example train data: ", x_train[17], y_train[17])

Example train data:  [0 0 0 0 1 0 0 0 0] True


In [22]:
from tqdm import tqdm
# Define the optimizer
from qiskit.algorithms.optimizers import SPSA
optimizer = SPSA(maxiter = 100)

# Define the cost function to be minimized by the optimizer
def cost_function(params):

    cost = 0
    estimator = Estimator()
    observables = (
        SparsePauliOp("IIIIZIIII")
    )
    
    for x, y in zip(x_train, y_train):
        # Create a Quantum Circuit acting on a quantum register of nine qubits
        circ = make_circuit(x, params, layers)        

        circuits = (
            circ,
        )

        job = estimator.run(circuits, observables)
        result = job.result().values[0]
        
        prediction = (result + 1)/2
        
        cost += l2_loss(prediction, y)
        
    print(cost)
    return cost

# Initialize the parameters
params = np.random.rand(34*layers)*2*np.pi

# Train the circuit
print('Initial parameters:', params)

# Check the qiskit docs to figure out how to start an optimizer
result = optimizer.minimize(cost_function, x0 = params)
print("Finished optimization.")

Initial parameters: [4.59780132 4.57131914 1.41245772 2.07085177 0.31254907 0.61138615
 4.71843809 1.07638506 5.01716295 2.94441069 4.35481884 3.33928748
 2.82734077 5.38864014 1.06570921 5.47167367 0.38992287 0.44414865
 1.95215839 2.0239641  3.13140018 5.31295428 0.48937321 2.15480056
 0.22585649 3.86862989 5.35571912 4.9769219  2.96920391 1.75357277
 1.43030443 4.53721451 6.2739144  3.49657467 5.4593263  5.78461774
 2.65950056 5.17888941 0.38334496 4.65528736 5.69879854 4.18679108
 3.54702697 2.73432631 6.12561718 4.78343464 2.85925149 1.74091417
 0.14012911 2.00896597 5.01222682 2.67520317 3.88967207 1.5991458
 3.94566209 4.07710621 0.88176167 3.05817652 2.72186822 0.47972665
 4.9708576  1.96427166 5.927205   0.33167786 0.40034846 2.6297153
 3.37873821 4.00377439 3.50289581 3.88141186 1.24511997 5.77988394
 2.80862644 1.97709428 3.43403897 3.13292944 4.94474788 5.70735744
 2.07328629 2.7553512  3.4403538  2.49427137 0.23301397 1.7657486
 1.58169763 4.56337888 4.99622638 3.3638885  

In [23]:
def predict(data, params):
    circ = make_circuit(data, params, layers)

    #TODO:  create observables and measure

    estimator = Estimator()
    observables = (
        SparsePauliOp("IIIIZIIII")
    )
    
    circuits = (
            circ
        )

    job = estimator.run(circuits, observables)
    result = job.result().values[0]
    
    

    return (result + 1)/2

In [24]:
def test_model(params, x_data, y_data):
    total_loss = 0
    correct_guesses = 0
    i = 0
    for x, y in zip(x_data, y_data):
        pred = predict(x, params)
        pred_discrete = round(pred)
        total_loss += l2_loss(pred, y)
        correct_guesses += np.all(pred_discrete == y)
        
        if i % 200 == 0 and False: 
            print("Correct output:", y)
            print("Actual output:", pred)
            print("Discrete_output:", pred_discrete)
            print("---------------")

    total_loss = total_loss / len(x_data)

    print("Correct guesses: {}/{} ({}%)".format(correct_guesses, len(x_data), round(correct_guesses/len(x_data)*100)))
    print("L2 Loss: {}".format(total_loss))
    
print("Train data:")
test_model(result.x, x_train, y_train)

print("Test data:")
test_model(result.x, x_test, y_test)

Train data:
Correct guesses: 84/132 (64%)
L2 Loss: 0.24012051931492495
Test data:
Correct guesses: 158/308 (51%)
L2 Loss: 0.2486558865799167
